In [1]:
%load_ext autoreload
%autoreload 2

# Paramètres de la pipeline

In [2]:
code_departement = "093"
logs_level = "WARNING"

In [3]:
# Parameters
code_departement = "093"
logs_level = "WARNING"


# Imports & setup

In [4]:
import geopandas as gpd

from potentiel_solaire.attach_buildings_to_schools import attach_buildings_to_schools
from potentiel_solaire.constants import DEFAULT_CRS, ALGORITHME_FOLDER, DATA_FOLDER
from potentiel_solaire.sources.bd_topo import extract_bd_topo, get_topo_zones_of_interest, \
    get_topo_buildings_of_interest
from potentiel_solaire.sources.bd_pci import extract_bd_pci
from potentiel_solaire.sources.bd_solar_irradiation import extract_bd_irradiation
from potentiel_solaire.sources.schools_establishments import extract_schools_establishments, \
    get_schools_establishments_of_interest
from potentiel_solaire.sources.protected_buildings import extract_protected_buildings, get_areas_with_protected_buildings
from potentiel_solaire.features.solar_potential import calculate_solar_potential
from potentiel_solaire.aggregate import aggregate_solar_potential_by
from potentiel_solaire.logger import get_logger

logger = get_logger()
logger.setLevel(logs_level)

2025-03-09 20:09:44,321 - DEBUG - rasterio.session - /Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/rasterio/session.py - <module> - Could not import boto3, continuing with reduced functionality.
2025-03-09 20:09:44,323 - DEBUG - rasterio.env - /Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/rasterio/env.py - <module> - GDAL data found in package: path='/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/rasterio/gdal_data'.
2025-03-09 20:09:44,324 - DEBUG - rasterio.env - /Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/rasterio/env.py - <module> - PROJ data found in package: path='/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/rasterio/proj_data'.


# Extraction des données sources

### Etablissements scolaires

In [5]:
schools_establishments_path = extract_schools_establishments()
print(f"Annuaire des établissements scolaires extrait ici: {schools_establishments_path}")

Annuaire des établissements scolaires extrait ici: /Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/data/fr-en-annuaire-education.geojson


### BD TOPO

In [6]:
bd_topo_path = extract_bd_topo(code_departement=code_departement)
print(f"BD TOPO extraite ici: {bd_topo_path}")

BD TOPO extraite ici: /Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/data/BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15/BDTOPO/1_DONNEES_LIVRAISON_2024-12-00134/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg


### BD PCI

In [7]:
bd_pci_path = extract_bd_pci(code_departement=code_departement)
print(f"BD PCI extraite ici: {bd_pci_path}")

BD PCI extraite ici: /Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/data/PARCELLAIRE-EXPRESS_1-1__SHP_LAMB93_D093_2025-01-01/PARCELLAIRE-EXPRESS/1_DONNEES_LIVRAISON_2025-02-00010/PEPCI_1-1_SHP_LAMB93_D093/BATIMENT.SHP


### BD IRRADIATION

In [8]:
bd_irradiation_path = extract_bd_irradiation()
print(f"BD irradiation extraite ici: {bd_irradiation_path}")

BD irradiation extraite ici: /Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/data/ENR_1-0_IRR-SOL_TIFF_WGS84G_FXX_2023-10-01/1_DONNEES_LIVRAISON/GlobalHorizontalIrradiation.tif


### BD Protected Buildings

In [9]:
bd_protected_buildings_path = extract_protected_buildings()
print(f"BD des bâtiments protégés extraite ici: {bd_protected_buildings_path}")

2025-03-09 20:10:04,889 - WARNING - pyogrio._io - /Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/pyogrio/raw.py - read - Skipping field cog_insee_lors_de_la_protection: unsupported OGR type: 5
2025-03-09 20:10:04,890 - WARNING - pyogrio._io - /Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/pyogrio/raw.py - read - Skipping field region: unsupported OGR type: 5
2025-03-09 20:10:04,890 - WARNING - pyogrio._io - /Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/pyogrio/raw.py - read - Skipping field departement_en_lettres: unsupported OGR type: 5


BD des bâtiments protégés extraite ici: /Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/data/liste_immeubles_proteges.geojson


# Filtre des données sur le périmètre du calcul

### Etablissements scolaires


In [10]:
schools_establishments = get_schools_establishments_of_interest(
    schools_filepath=schools_establishments_path,
    code_departement=code_departement,
    types_etablissements=['Ecole', 'Lycée', 'Collège'],
    statut_public_prive="Public",
    etat="OUVERT",
    crs=DEFAULT_CRS
)
nb_schools = schools_establishments.shape[0]
print(f"Nb d'établissements scolaires: {nb_schools}")

Nb d'établissements scolaires: 1130


### Zone d'intérêt géographique

In [11]:
codes_commune = schools_establishments["code_commune"].unique()
communes = gpd.read_file(bd_topo_path, layer="commune").to_crs(DEFAULT_CRS)
communes = communes[communes.code_insee.isin(codes_commune)]
geom_of_interest = communes.dissolve()[["geometry"]]

### Zones d'éducations

In [12]:
educational_zones = get_topo_zones_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    categories=["Science et enseignement"],
    natures=['Collège', 'Lycée', 'Enseignement primaire'],
    crs=DEFAULT_CRS
)
nb_educational_zones = educational_zones.shape[0]
print("Nb de zones d'éducations: ", nb_educational_zones)

/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/pyogrio/core.py:279: RuntimeWarning: Field format 'character varying' not supported
  return ogr_read_info(
/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/pyogrio/core.py:279: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_read_info(
/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/pyogrio/core.py:279: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_read_info(
/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/pyogrio/core.py:279: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_read_info(
/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/geopandas/io/file.py:497: UserWarning: More than one layer found in 'BDT_3-4_GPKG_

Nb de zones d'éducations:  1088


### Bâtiments

In [13]:
# TODO : ajout des batiments manquants avec la BD PCI
buildings = get_topo_buildings_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    crs=DEFAULT_CRS
)
nb_buildings = buildings.shape[0]
print("Nb de batiments: ", nb_buildings)

/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/pyogrio/core.py:279: RuntimeWarning: Field format 'character varying' not supported
  return ogr_read_info(
/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/pyogrio/core.py:279: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_read_info(
/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/pyogrio/core.py:279: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_read_info(
/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/pyogrio/core.py:279: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_read_info(
/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/geopandas/io/file.py:497: UserWarning: More than one layer found in 'BDT_3-4_GPKG_

Nb de batiments:  351577


### Zones avec des bâtiments protégés

In [14]:
areas_with_protected_buildings = get_areas_with_protected_buildings(
    bd_protected_buildings_path=bd_protected_buildings_path,
    geom_of_interest=geom_of_interest,
    crs=DEFAULT_CRS
)

# Détermination des bâtiments scolaires

In [15]:
schools_buildings = attach_buildings_to_schools(
    schools_establishments=schools_establishments,
    educational_zones=educational_zones,
    buildings=buildings
)
nb_schools_buildings = schools_buildings.shape[0]
print("Nb de batiments scolaires: ", nb_schools_buildings)

/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/.venv/lib/python3.13/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


Nb de batiments scolaires:  3731


In [16]:
schools_buildings.head()

,cleabs_bat,construction_legere,hauteur,geometry,index_zone,cleabs_zone,categorie,nature,toponyme,identifiant_de_l_etablissement
4,BATIMENT0000000229474428,False,24.7,"MULTIPOLYGON Z (((2.3241 48.90492 44.5, 2.3240...",409,SURFACTI0000000002555886,Science et enseignement,Enseignement primaire,École Maternelle Victor Hugo,0930409P
237,BATIMENT0000000243403398,False,7.2,"MULTIPOLYGON Z (((2.36008 48.9081 44.2, 2.3601...",418,SURFACTI0000000246200139,Science et enseignement,Enseignement primaire,École Élémentaire Louis Antoine Saint-Just,0931478B
238,BATIMENT0000000243403396,False,6.8,"MULTIPOLYGON Z (((2.36022 48.90797 43.8, 2.359...",418,SURFACTI0000000246200139,Science et enseignement,Enseignement primaire,École Élémentaire Louis Antoine Saint-Just,0931478B
245,BATIMENT0000000243403399,True,NaN,"MULTIPOLYGON Z (((2.36046 48.90831 -1000, 2.36...",418,SURFACTI0000000246200139,Science et enseignement,Enseignement primaire,École Élémentaire Louis Antoine Saint-Just,0931478B
253,BATIMENT0000000243403625,False,NaN,"MULTIPOLYGON Z (((2.36118 48.908 -1000, 2.3612...",418,SURFACTI0000000246200139,Science et enseignement,Enseignement primaire,École Élémentaire Louis Antoine Saint-Just,0931478B


# Calcul des attributs utiles pour le potentiel solaire

In [17]:
# TODO: v0 seulement à ce stade
solar_potential_of_schools_buildings = calculate_solar_potential(
    schools_buildings=schools_buildings,
    bd_irradiation_path=bd_irradiation_path,
    areas_with_protected_buildings=areas_with_protected_buildings
)

solar_potential_of_schools_buildings

/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/potentiel_solaire/features/solar_potential.py:36: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  schools_buildings['lon'] = schools_buildings.centroid.map(lambda p: p.x)
/Users/ASalles/dataforgood/13_potentiel_solaire/algorithme/potentiel_solaire/features/solar_potential.py:37: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  schools_buildings['lat'] = schools_buildings.centroid.map(lambda p: p.y)


,cleabs_bat,construction_legere,hauteur,geometry,index_zone,cleabs_zone,categorie,nature,toponyme,identifiant_de_l_etablissement,lon,lat,protection,surface_totale_au_sol,zone_pour_rayonnement_solaire,surface_utile,peakpower,url,productible
4,BATIMENT0000000229474428,False,24.7,"MULTIPOLYGON Z (((2.3241 48.90492 44.5, 2.3240...",409,SURFACTI0000000002555886,Science et enseignement,Enseignement primaire,École Maternelle Victor Hugo,0930409P,2.324039,48.904789,False,390.546688,"POLYGON ((2.31409 48.92563, 2.31408 48.92565, ...",90.311475,20.320082,https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?&l...,22.39781
237,BATIMENT0000000243403398,False,7.2,"MULTIPOLYGON Z (((2.36008 48.9081 44.2, 2.3601...",418,SURFACTI0000000246200139,Science et enseignement,Enseignement primaire,École Élémentaire Louis Antoine Saint-Just,0931478B,2.360569,48.908249,True,2169.036390,"POLYGON ((2.35849 48.88451, 2.35819 48.88453, ...",1301.421834,292.819913,https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?&l...,320.43220
238,BATIMENT0000000243403396,False,6.8,"MULTIPOLYGON Z (((2.36022 48.90797 43.8, 2.359...",418,SURFACTI0000000246200139,Science et enseignement,Enseignement primaire,École Élémentaire Louis Antoine Saint-Just,0931478B,2.360053,48.908029,True,312.978909,"POLYGON ((2.35737 48.93155, 2.35763 48.9316, 2...",70.432246,15.847255,https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?&l...,17.34157
245,BATIMENT0000000243403399,True,NaN,"MULTIPOLYGON Z (((2.36046 48.90831 -1000, 2.36...",418,SURFACTI0000000246200139,Science et enseignement,Enseignement primaire,École Élémentaire Louis Antoine Saint-Just,0931478B,2.360468,48.908307,False,0.990138,"POLYGON ((2.36037 48.93199, 2.36038 48.93199, ...",0.000000,0.000000,None,0.00000
253,BATIMENT0000000243403625,False,NaN,"MULTIPOLYGON Z (((2.36118 48.908 -1000, 2.3612...",418,SURFACTI0000000246200139,Science et enseignement,Enseignement primaire,École Élémentaire Louis Antoine Saint-Just,0931478B,2.361212,48.908000,False,6.420895,"POLYGON ((2.34099 48.91337, 2.341 48.91338, 2....",0.000000,0.000000,None,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348076,BATIMENT0000000246439599,True,NaN,"MULTIPOLYGON Z (((2.58408 48.94175 -1000, 2.58...",306,SURFACTI0000000002555563,Science et enseignement,Collège,Collège Romain Rolland,0931191P,2.584090,48.941770,False,7.330652,"POLYGON ((2.56534 48.93164, 2.56533 48.93167, ...",0.000000,0.000000,None,0.00000
348077,BATIMENT0000000246439600,True,NaN,"MULTIPOLYGON Z (((2.58405 48.9419 -1000, 2.584...",306,SURFACTI0000000002555563,Science et enseignement,Collège,Collège Romain Rolland,0931191P,2.584104,48.941834,False,46.494143,"POLYGON ((2.56572 48.93085, 2.56488 48.9329, 2...",0.000000,0.000000,None,0.00000
348090,BATIMENT0000000246440832,False,NaN,"MULTIPOLYGON Z (((2.58372 48.94178 -1000, 2.58...",306,SURFACTI0000000002555563,Science et enseignement,Collège,Collège Romain Rolland,0931191P,2.583765,48.941744,False,37.523334,"POLYGON ((2.57996 48.96508, 2.58001 48.96509, ...",0.000000,0.000000,None,0.00000
348100,BATIMENT0000002244515389,False,NaN,"MULTIPOLYGON Z (((2.58291 48.94149 -1000, 2.58...",306,SURFACTI0000000002555563,Science et enseignement,Collège,Collège Romain Rolland,0931191P,2.583871,48.941507,False,2097.830515,"POLYGON ((2.56302 48.93493, 2.56286 48.93549, ...",1258.698309,283.207120,https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?&l...,308.00684


# Dump des donnees pour analyses

In [ ]:
# TODO: Finir la pipeline à partir d'ici.
solar_potential_of_schools_buildings["zone_pour_rayonnement_solaire"] = solar_potential_of_schools_buildings["zone_pour_rayonnement_solaire"].to_wkt()

layers = ["schools_establishments", "educational_zones", "schools_buildings", "solar_potential_of_schools_buildings"]
gdfs = [schools_establishments, educational_zones, schools_buildings, solar_potential_of_schools_buildings]

for layer, gdf in zip(layers, gdfs):
    output_gpkg = DATA_FOLDER / f"{code_departement}_pipeline_results.gpkg"
    gdf.to_file(output_gpkg, layer=layer, driver="GPKG")

# Checks sur la qualité des données & calculs

In [23]:
nb_schools_with_buildings = len(schools_buildings.identifiant_de_l_etablissement.unique())
print("Nb d'établissements scolaires avec des batiments: {} ({}%)".format(
    nb_schools_with_buildings,
    round(100 * nb_schools_with_buildings / nb_schools)
))

Nb d'établissements scolaires avec des batiments: 445 (39%)


In [24]:
nb_buildings_protected = solar_potential_of_schools_buildings[solar_potential_of_schools_buildings.protection].shape[0]
print(f"Nb de batiments protégés {nb_buildings_protected} ({100*nb_buildings_protected/nb_schools_buildings:.2f}%)")

Nb de batiments protégés 1093 (29.30%)


# Aggrégations

### Par établissement scolaire

In [21]:
results_by_school = aggregate_solar_potential_by(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings,
    group_by = [
        "identifiant_de_l_etablissement",
        "nom_etablissement",
        "type_etablissement",
        "libelle_nature",
        "code_commune",
        "nom_commune",
        "code_departement",
        "libelle_departement",
        "code_region",
        "libelle_region",
    ]
)

KeyError: "Column(s) ['potentiel_solaire', 'rayonnement_solaire'] do not exist"

### Par commune

In [ ]:
results_by_commune = aggregate_solar_potential_by(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings,
    group_by = [
        "code_commune",
        "nom_commune",
        "code_departement",
        "libelle_departement",
        "code_region",
        "libelle_region",
    ]
)
# TODO : la geometrie de la commune est fausse

### Par département

In [ ]:
results_by_departement = aggregate_solar_potential_by(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings,
    group_by = [
        "code_departement",
        "libelle_departement",
        "code_region",
        "libelle_region",
    ]
)
# TODO : la geometrie du departement est fausse

### Par région

In [ ]:
# TODO : demande de se connecter au bucket avec les resultats existants pour aggreger 
# TODO : ou de faire le calcul pour tous les departements d une region

# Sauvegarde des fichiers

In [ ]:
output_folder = ALGORITHME_FOLDER.parent / "results" / f"D{code_departement}"
output_folder.mkdir(exist_ok=True, parents=True)

# TODO : a voir cote front sil faut changer le format de mise a disposition
results_by_school.to_file(output_folder / f"D{code_departement}_ecoles.geojson", driver="GeoJSON")
results_by_commune.to_file(output_folder / f"D{code_departement}_communes.geojson", driver="GeoJSON")
results_by_departement.to_file(output_folder / f"D{code_departement}_departement.geojson", driver="GeoJSON")